In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from IPython import display

np.random.seed(0)

# save the trajectory into a file
import pickle
import os
# find the next available file path
def get_next_filename(base_name, extension):
    index = 1
    while True:
        filename = f"{base_name}_{index}.{extension}"
        if not os.path.exists(filename):
            return filename
        index += 1

# find the next available folder
def get_next_folder(base_name):
    index = 1
    while True:
        folder = f"{base_name}_{index}"
        if not os.path.exists(folder):
            return folder
        index += 1

# create a folder called world
folder = get_next_folder("world")
os.mkdir(folder)

In [ ]:
from grid_env import GridEnv
env = GridEnv(sparse=True)
env.plot()

In [ ]:
#Create new env with estimated reward and train agent
new_env = GridEnv(move_penalty=0.0, collision_penalty=0.0)
newQ = new_env.value_iter2(iters=150000, alpha=0.5,gamma=0.99,initial_eps=1.0,eps_anneal_rate=0.7,disp=True,plot_freq=1000)

In [ ]:
# save the new Q as a pickle file
filename = f"{folder}/Q_table.pkl"
with open(filename, "wb") as f:
    pickle.dump(newQ, f)

In [ ]:
# Test agent on original world
rsum,traj = new_env.test_value(newQ,steps=100,disp=True, greediness=0.8)

In [ ]:
# collect trajectories from 1000 episodes
trajs = []
for i in range(3000):
    rsum,traj = new_env.test_value(newQ,steps=400,disp=False, greediness=1)
    trajs.append(np.vstack(traj))
trajs = np.array(trajs,dtype=object)

In [ ]:
Q_image = np.max(newQ,axis=1).reshape(env.N,env.N).T

num_plot = 5
fig, axs = plt.subplots(1, num_plot, figsize=(20,5))
# plot each state pair
for i in range(num_plot):
    # plot the first state
    axs[i].imshow(env.occ_map.T, origin="lower",extent=[0,env.N,0,env.N], cmap='gray')
    axs[i].plot(env.xl+0.5,env.yl+0.5,'gx') # goal location
    axs[i].plot(trajs[i][:,0]+0.5,trajs[i][:,1]+0.5, '-o',label=f'trajectory {i+1}')
    # finish up the rest of the plot settings
    axs[i].legend()
    axs[i].axis('off')
plt.show()

In [ ]:

# save the trajectories into that folder
filename = f"{folder}/trajs.pkl"
# save the trajectories into that folder
with open(filename, "wb") as f:
    pickle.dump(trajs, f)
